In [1]:
import os
import datetime
import json
import tweepy as tw
import pandas as pd  
import numpy as np
from urllib.request import urlopen
import requests
from neo4j import __version__ as neo4j_version
from neo4j import GraphDatabase
import spacy
from spacy.tokens import DocBin
from bcr_api.bwproject import BWProject, BWUser
from bcr_api.bwresources import BWQueries, BWGroups, BWAuthorLists, BWSiteLists, BWLocationLists, BWTags, BWCategories, BWRules, BWMentions, BWSignals
from random import randint
import json

In [2]:
# Function to create an entity dictionary from spaCy's NER
def get_ent_list(doc):
    
    entList = []
    
    for ent in doc.ents:
        if ent.label_ == 'PERSON' or ent.label_ == 'ORG' or ent.label_ == 'EVENT' or ent.label_ == 'FAC':
            entList.append(ent.text)
        else:
            pass

    return entList

# Function to input language code from Twitter and return appropriate Spacy model
def getModelName(langCode):
    switcher = {
        'da': "da_core_news_md",
        'de': "de_core_news_md",
        'en': "en_core_web_md",
        'es': "es_core_news_md",
        'fr': "fr_core_news_md",
        'it': "it_core_news_md",
        'no': "nb_core_news_md",
        'nl': "nl_core_news_md",
        'pl': "pl_core_news_md",
        'pt': "pt_core_news_md",
        'ro': "ro_core_news_md",
        'ru': "ru_core_news_md",
        'zh': "zh_core_web_md"
    }
    return switcher.get(langCode, "unknown")

# Function to get entities based on inputting language and text
def getEntities(language, text):
    model = getModelName(language)
    
    if model != 'unknown':
        nlp = spacy.load(model)
        doc = nlp(text)
        entList = get_ent_list(doc)
        return entList
    else:
        entList = []
        return entList


In [3]:
#Define the Neo4j connection class

class Neo4jConnection:
    
    def __init__(self, uri, user, pwd):
        self.__uri = uri
        self.__user = user
        self.__pwd = pwd
        self.__driver = None
        try:
            self.__driver = GraphDatabase.driver(self.__uri, auth=(self.__user, self.__pwd))
        except Exception as e:
            print("Failed to create the driver:", e)
        
    def close(self):
        if self.__driver is not None:
            self.__driver.close()
        
    def query(self, query, db=None):
        assert self.__driver is not None, "Driver not initialized!"
        session = None
        response = None
        try: 
            session = self.__driver.session(database=db) if db is not None else self.__driver.session() 
            response = list(session.run(query))
        except Exception as e:
            print("Query failed:", e)
        finally: 
            if session is not None:
                session.close()
        return response

In [4]:
# Function to identify tweet type
def getTweetType(tweet):
    try:
        if ((tweet.in_reply_to_status_id is None) and (tweet.retweeted_status is None) and (tweet.is_quote_status == False)):
            typeT = "Regular"
        elif tweet.is_quote_status == True:
            typeT = "Quote tweet"    
        elif tweet.in_reply_to_status_id is not None:
            typeT = "Reply"
        elif tweet.retweeted_status is not None:
            typeT = "Retweeted"        
        else:
            typeT = "Unknown"
    except:
        if ((tweet.in_reply_to_status_id is None) and (tweet.is_quote_status == False)):
            typeT = "Regular"
        elif tweet.is_quote_status == True:
            typeT = "Quote tweet"    
        elif tweet.in_reply_to_status_id is not None:
            typeT = "Reply"        
        else:
            typeT = "Unknown"        
    return(typeT)



In [5]:
### CONNECTING TO BRANDWATCH API ###

# Get login and password from file 
f=open("login.txt","r")
lines=f.readlines()

BWusername=lines[0]
BWusername=BWusername.rstrip()
BWpassword=lines[1]
f.close()

# Direct Brandwatch to a separate tokens file that's required
BWUser(username=BWusername, password=BWpassword, token_path="tokens.txt")

# Connect to the API
YOUR_ACCOUNT = BWusername
YOUR_PROJECT = "Cargill"
project = BWProject(username=YOUR_ACCOUNT, project=YOUR_PROJECT)

09:12:56 INFO: Overwriting access token for james_warden@cargill.com in tokens.txt
09:12:57 DEBUG: https://api.brandwatch.com/projects


In [6]:
### CONNECTING TO TWITTER API ###

# Get keys from file
f=open("twitter-keys.txt","r")
lines=f.readlines()
consumer_key=lines[0].rstrip()
consumer_secret = lines[1].rstrip()
access_token = lines[2].rstrip()
access_token_secret = lines[3].rstrip() 
f.close()

# Connect to the API
auth = tw.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tw.API(auth, wait_on_rate_limit=True)

In [7]:
### CONNECTING TO NEO4J SERVER ###

# Note that this is running locally on my laptop
# Server must be running for any Neo4j interactions to work

# Get uri, login and password from file 
f=open("neo4j.txt","r")
lines=f.readlines()
uri = lines[0].rstrip()
NEOusername=lines[1].rstrip()
NEOpassword=lines[2].rstrip()
f.close()

# Connect to Neo4j
conn = Neo4jConnection(uri="bolt://localhost:7687", user=NEOusername, pwd=NEOpassword)

In [ ]:
today = '2021-11-22'
start = '2021-10-22'

queries = BWQueries(project)
filtered = queries.get_mentions(name = "Cargill Main",
                                startDate = start, 
                                endDate = today)

In [ ]:
# Putting this in there in case I messed up in some way
with open('brandwatch_save.json', 'w') as saving:
    json.dump(filtered , saving)

In [8]:
f = open('brandwatch_save.json',)
filtered = json.load(f)

In [9]:
counter = 0 

# Parent categories that are the only ones I'll pull tags from
relevantCats = ['Activists', 'Cargill Brands', 'Cargill Leaders', 'Competitors', 'Customers', 'Customer & Competitor Executives', 
                'Events', 'Financial Institutions', 'Labor', 'Metals Competitors', 'Other Companies', 'Other Organizations', 
                'Parties', 'Partners', 'Politicians', 'Trade Orgs']

for f in filtered:
    print(counter)
    irrelevant = False # set irrelevant to false for use later
    
    categoriesDict = f['categoryDetails']
    categoriesList = [] # Categories captured through boolean tags in the platform
    
    for c in categoriesDict:
        if c['parentName'] == 'Irrelevant':
            irrelevant = True # change irrelevant to true so we don't use this post
            break # Get out of the interior for loop because it's no longer necessary if there's an irrelevant tag
            
        if c['parentName'] in relevantCats:
            categoriesList.append(c['name'])
    
    if irrelevant is False: # This says skip process if it's an irrelevant item
        pageType = f['pageType']
        url = f['url']
        date = f['date']

        try:
        
            # We need authors for some of our nodes but some are blank or unknown
            # We're creating a random name for them
            author = f['author']
            if author is None or len(author) == 0:
                authID = randint(1, 999999999) 
                authID = str(authID)
                author = "unknown"+authID

            country = str(f['country'])
            text = f['fullText']
            text = str(text) # converts none type to a string for CQL
            text = text.replace('"',"")
            
            language = f['language']
            domain = f['domain']


            if pageType == 'twitter': # Twitter doesn't come through with text, so we'll process it differently

                tweetID = f['guid'] # Grab ID that will be used for Twitter API pull
                tweet = api.get_status(tweetID, tweet_mode = "extended") # Grab the tweet object via Tweepy
                tweetType = getTweetType(tweet) # Determine tweet type
                print(tweetType)

                # Get user information
                screen_name = tweet.user.screen_name
                name = tweet.user.name
                userID = str(tweet.user.id_str)
                userDescript = str(tweet.user.description)
                followers = str(tweet.user.followers_count)
                location = str(tweet.user.location)
                verified = str(tweet.user.verified)

                ### Merge author node ###
                query_string = "MERGE (e:Entity {ID:\""+userID+"\", Type:\"TwitterAccount\", Name:\'"+screen_name+"\'})"
                conn.query(query_string, db='neo4j')
                query_string = "MATCH (e:Entity {ID:\""+userID+"\", Type:\"TwitterAccount\", Name:\'"+screen_name+"\'}) SET e.Description = \"" + userDescript +"\", e.Followers ="+followers+", e.Location = \""+location+"\", e.Verified = "+verified
                conn.query(query_string, db='neo4j')

                ### Merge tweet node placeholder ###
                query_string = "MERGE (p:Post {ID:\""+tweetID+"\", Type:\"Tweet\"})"
                conn.query(query_string, db='neo4j')

                if tweetType == 'Retweeted':

                    # Because retweets are just sharing standalone tweets, we're not doing further processing
                    # Instead, we're just going to create the relationship to the node that was merged above.                 
                    # If the tweet data hasn't been set, the above will remain a placeholder node that will be processed when the original comes through
                    # If the tweet did exist, there's no further data to provide.
                    # This cuts processing time.

                    # Create retweet edge
                    query_string = "MATCH (e:Entity), (p:Post) WHERE e.ID = \""+userID+"\" AND p.ID = \""+tweetID+"\" AND p.Type = \"Tweet\" CREATE (e)-[r:RETWEETED]->(p)"
                    conn.query(query_string, db='neo4j')

                else:



                    # Gathering information for special edges for @mentions 
                    mentionsDict = tweet.entities['user_mentions'] # Grab data on all accounts @mentioned
                    mentionsList = [] # Only being used for checks to avoid NER causing duplicates

                    for m in mentionsDict:
                        mentionsList.append(m['screen_name']) # Build out the list

                        if m['screen_name'] in categoriesList:
                            categoriesList.remove(m['screen_name']) # Remove copies in categoriesList to avoid duplicates

                    text = tweet.full_text # Get the full text from the tweet object
                    text = str(text)
                    text = text.replace('"',"")
                    
                    TWdate = tweet.created_at 

                    # NER Extraction
                    extractedEntities = getEntities(language, text) # Get additional entities through NER
                    if extractedEntities != 'noModel' or len(extractedEntities) > 0: # If NER found entities ...
                        for e in extractedEntities: # Loop through the entities ...
                            if e not in categoriesList:
                                categoriesList.append(e) # and add them to the categories list if they're not already there                

                    # Add date and text
                    query_string = "MATCH (p:Post {ID:\""+tweetID+"\", Type:\"Tweet\"}) SET p.Date = \""+TWdate.isoformat()+"\", p.Text = \""+text+"\""
                    conn.query(query_string, db='neo4j')

                    # Create mentions relationship
                    for m in mentionsDict:
                        M_screen_name = m['screen_name']
                        M_id = str(m['id'])

                        # Merge nodes
                        query_string = "MERGE (e:Entity {ID:\""+M_id+"\", Type:\"TwitterAccount\", Name:\'"+M_screen_name+"\'})"
                        conn.query(query_string, db='neo4j')

                        # Create relationships
                        query_string = "MATCH (p:Post), (e:Entity) WHERE p.ID = \""+tweetID+"\" AND e.ID = \""+M_id+"\" CREATE (p)-[r:AT_MENTIONED]->(e)"
                        conn.query(query_string, db='neo4j')

                    # Create categories list nodes
                    for c in categoriesList:

                        # Merge nodes
                        query_string = "MERGE (e:Entity {Type:\"autoID\", Name:\'"+c+"\'})"
                        conn.query(query_string, db='neo4j')

                        # Create relationship
                        query_string = "MATCH (p:Post), (e:Entity) WHERE p.ID = \""+tweetID+"\" AND e.Name = \""+c+"\" AND e.Type = \"autoID\" CREATE (p)-[r:MENTIONED]->(e)"
                        conn.query(query_string, db='neo4j')                    

                    if tweetType == 'Quote tweet':

                        originalID = tweet.quoted_status_id_str

                        # Create placeholder node for original tweet
                        query_string = "MERGE (p:Post {ID:\""+originalID+"\", Type:\"Tweet\"})"
                        conn.query(query_string, db='neo4j')

                        # Create quote-original relationship
                        query_string = "MATCH (p:Post), (o:Post) WHERE p.ID = \""+tweetID+"\" AND o.ID = \""+originalID+"\" AND p.Type = \"Tweet\" AND o.Type = \"Tweet\" CREATE (p)-[r:QUOTE_TWEETED]->(o)"
                        conn.query(query_string, db='neo4j') 

                        # Create entity-tweet relationship
                        query_string = "MATCH (e:Entity), (p:Post) WHERE e.ID = \""+userID+"\" AND p.ID = \""+tweetID+"\" AND p.Type = \"Tweet\" CREATE (e)-[r:TWEETED]->(p)"
                        conn.query(query_string, db='neo4j')

                    elif tweetType == 'Reply':

                        originalID = tweet.in_reply_to_status_id_str

                        # Create placeholder node for original tweet
                        query_string = "MERGE (p:Post {ID:\""+originalID+"\", Type:\"Tweet\"})"
                        conn.query(query_string, db='neo4j')

                        # Create reply-original relationship
                        query_string = "MATCH (p:Post), (o:Post) WHERE p.ID = \""+tweetID+"\" AND p.ID = \""+originalID+"\" AND p.Type = \"Tweet\" CREATE (p)-[r:REPLIED_TO]->(o)"
                        conn.query(query_string, db='neo4j')

                        # Create account-tweet relationship
                        query_string = "MATCH (e:Entity), (p:Post) WHERE e.ID = \""+userID+"\" AND p.ID = \""+tweetID+"\" AND p.Type = \"Tweet\" CREATE (e)-[r:TWEETED]->(p)"
                        conn.query(query_string, db='neo4j')

                    elif tweetType == 'Regular':

                        # Create account-tweet relationship
                        query_string = "MATCH (e:Entity), (p:Post) WHERE e.ID = \""+userID+"\" AND p.ID = \""+tweetID+"\" AND p.Type = \"Tweet\" CREATE (e)-[r:TWEETED]->(p)"
                        conn.query(query_string, db='neo4j')


                    else:

                        # We already took care of simple retweets 
                        # We've also already taken care of all connections for regular tweets
                        # So there's nothing left to do 

                        pass

            else:

                # Get additional entities through NER
                # Even with types that don't provide full text, maybe we'll get lucky
                extractedEntities = getEntities(language, text) 

                # If additional entities were found, add them into the categories list
                if extractedEntities != 'noModel' or len(extractedEntities) > 0: 
                    for e in extractedEntities:
                        if e not in categoriesList:
                            categoriesList.append(e)

                if pageType == 'news' or pageType == 'blog':
                    domain = str(f['domain'])
                    UVs = str(f['monthlyVisitors'])
                    title = str(f['title'])

                    # Merge publication node
                    query_string = "MERGE (e:Entity {Name:\'"+domain+"\', Type:\""+pageType+"\"})"                
                    conn.query(query_string, db='neo4j')

                    query_string = "MATCH (e:Entity {Name:\'"+domain+"\', Type:\""+pageType+"\"}) SET e.Visitors = "+UVs+", e.Location =\""+country+"\""
                    conn.query(query_string, db='neo4j')

                    # Merge post node
                    query_string = "MERGE (p:Post {Type:\""+pageType+"\", Name:\""+title+"\", Date:\""+date+"\", Link:\""+url+"\", Language:\""+language+"\", Text:\""+text+"\"})"
                    conn.query(query_string, db='neo4j')

                    # Create relationship
                    query_string = "MATCH (e:Entity), (p:Post) WHERE e.Name = \""+domain+"\" AND e.Type = \""+pageType+"\" AND p.Link = \""+url+"\" CREATE (e)-[r:PUBLISHED]->(p)"
                    conn.query(query_string, db='neo4j')

                    # Create categories list nodes
                    for c in categoriesList:

                        # Merge nodes
                        query_string = "MERGE (e:Entity {Type:\"autoID\", Name:\'"+c+"\'})"
                        conn.query(query_string, db='neo4j')

                        # Create relationship
                        query_string = "MATCH (p:Post), (e:Entity) WHERE p.Link = \""+url+"\" AND p.Name = \""+title+"\" AND e.Type = \"autoID\" AND e.Name =\""+c+"\" CREATE (p)-[r:MENTIONED]->(e)"
                        conn.query(query_string, db='neo4j')

                elif pageType == 'instagram':
                    followers = str(f['instagramFollowerCount'])

                    # Brandwatch API does not provide Instagram authors for some reason, and ID is only for the post
                    # So we're just going to do post and mentions

                    # Merge post node
                    query_string = "MERGE (p:Post {Type:\""+pageType+"\", Date:\""+date+"\", Link:\""+url+"\", Language:\""+language+"\", Text:\""+text+"\"})"
                    conn.query(query_string, db='neo4j')

                    # Create categories list nodes
                    for c in categoriesList:

                        # Merge nodes
                        query_string = "MERGE (e:Entity {Type:\"autoID\", Name:\'"+c+"\'})"
                        conn.query(query_string, db='neo4j')

                        # Create relationship
                        query_string = "MATCH (p:Post), (e:Entity) WHERE p.Link = \""+url+"\" AND p.Date = \""+date+"\" AND e.Type = \"autoID\" AND e.Name =\""+c+"\" CREATE (p)-[r:MENTIONED]->(e)"
                        conn.query(query_string, db='neo4j')

                elif pageType == 'reddit':
                    subreddit = f['subreddit']
                    subSubscribers =f['subredditSubscribers']
                    title = str(f['title'])
                    thread = f['threadURL']

                    # Merge subreddit node
                    query_string = "MERGE (e:Entity {Name:\'"+subreddit+"\', Type:\"Subreddit\"})"                
                    conn.query(query_string, db='neo4j')                

                    # Merge subreddit thread
                    query_string = "MERGE (p:Post {Name:\'"+thread+"\', Type:\"SubredditThread\"})"                
                    conn.query(query_string, db='neo4j')               

                    # Merge author node
                    query_string = "MERGE (e:Entity {Name:\""+author+"\", Type:\"RedditAuthor\"})"       
                    conn.query(query_string, db='neo4j')

                    # Merge post node
                    query_string = "MERGE (p:Post {Type:\""+pageType+"Post\", Name:\""+title+"\", Date:\""+date+"\", Link:\""+url+"\", Language:\""+language+"\", Text:\""+text+"\"})"
                    conn.query(query_string, db='neo4j')

                    # Merge subreddit-thread relationship
                    query_string = "MATCH (e:Entity), (p:Post) WHERE e.Name = \""+subreddit+"\" AND e.Type = \"Subreddit\" AND p.Name = \""+thread+"\" AND p.Type = \"SubredditThread\" MERGE (e)-[r:PUBLISHED]->(p)"                
                    conn.query(query_string, db='neo4j')

                    # Merge thread-post relationship
                    query_string = "MATCH (p:Post), (t:Post) WHERE p.Name = \""+title+"\" AND p.Link = \""+url+"\" AND p.Type = \""+pageType+"Post\" AND t.Name = \""+thread+"\" AND t.Type = \"SubredditThread\" CREATE (p)-[r:REPLIED_TO]->(t)"                
                    conn.query(query_string, db='neo4j')

                    # Create author-post relationship
                    query_string = "MATCH (e:Entity), (p:Post) WHERE e.Name = \""+author+"\" AND e.Type = \"RedditAuthor\" AND p.Link = \""+url+"\" CREATE (e)-[r:AUTHORED]->(p)"
                    conn.query(query_string, db='neo4j')

                    # Create categories list nodes
                    for c in categoriesList:

                        # Merge nodes
                        query_string = "MERGE (e:Entity {Type:\"autoID\", Name:\'"+c+"\'})"
                        conn.query(query_string, db='neo4j')

                        # Create relationship
                        query_string = "MATCH (p:Post), (e:Entity) WHERE p.Link = \""+url+"\" AND p.Date = \""+date+"\" AND e.Type = \"autoID\" AND e.Name =\""+c+"\" CREATE (p)-[r:MENTIONED]->(e)"
                        conn.query(query_string, db='neo4j')

                elif pageType == 'youtube':

                    # Merge author node
                    query_string = "MERGE (e:Entity {Name:\'"+author+"\', Type:\"YouTubeAuthor\"})"                
                    conn.query(query_string, db='neo4j')

                    if f['threadEntryType'] == 'reply':

                        # Brandwatch doesn't provide the original video url as its own field for replies
                        # But the title field has it. We just have to remove part of the string
                        originalURL = filtered[597]['title']
                        originalURL = originalURL.replace('Comment on video ','')

                        # Merge original video
                        # Brandwatch doesn't provide many details on the original post with replies
                        # So there's not much more we can add
                        query_string = "MERGE (p:Post {Link:\""+originalURL+"\", Type:\""+pageType+"Original\"})"
                        conn.query(query_string, db='neo4j')

                        # Merge comment post
                        query_string = "MERGE (p:Post {Type:\""+pageType+"Reply\", Date:\""+date+"\", Link:\""+url+"\", Language:\""+language+"\", Text:\""+text+"\"})"
                        conn.query(query_string, db='neo4j')

                        # Create comment-original relationship
                        query_string = "MATCH (c:Post), (o:Post) WHERE c.Link = \""+url+"\" AND c.Type = \""+pageType+"Reply\" AND o.Link = \""+originalURL+"\" AND o.Type = \""+pageType+"Original\" CREATE (c)-[r:REPLIED_TO]->(o)" 
                        conn.query(query_string, db='neo4j')

                        # Create author-comment relationship
                        query_string = "MATCH (e:Entity), (c:Post) WHERE e.Name = \""+author+"\" AND e.Type = \"YouTubeAuthor\" AND c.Link = \""+url+"\" AND c.Type = \""+pageType+"\"Reply\" CREATE (e)-[r:AUTHORED]->(c)"
                        conn.query(query_string, db='neo4j')

                        # Create categories list nodes
                        for c in categoriesList:

                            # Merge nodes
                            query_string = "MERGE (e:Entity {Type:\"autoID\", Name:\'"+c+"\'})"
                            conn.query(query_string, db='neo4j')

                            # Create relationship
                            query_string = "MATCH (p:Post), (e:Entity) WHERE p.Link = \""+url+"\" AND p.Date =\""+date+"\" AND p.Type = \""+pageType+"Reply\" AND e.Type = \"autoID\" AND e.Name =\""+c+"\" CREATE (p)-[r:MENTIONED]->(e)"
                            conn.query(query_string, db='neo4j')

                    else:

                        # Merge original video
                        query_string = "MERGE (p:Post {Link:\""+url+"\", Type:\""+pageType+"Original\"})"
                        conn.query(query_string, db='neo4j')

                        query_string = "MATCH (P:Post {Link:\""+url+"\", Type:\""+pageType+"Original\"}) SET P.Date = \"" + date +"\", P.Text = \""
                        conn.query(query_string, db='neo4j')

                        # Create author-post relationship
                        query_string = "MATCH (e:Entity), (o:Post) WHERE e.Name = \""+author+"\" AND e.Type = \"YouTubeAuthor\" AND o.Link = \""+url+"\" AND o.Type = \""+pageType+"\"Original\" CREATE (e)-[r:AUTHORED]->(o)"
                        conn.query(query_string, db='neo4j')

                        # Create categories list nodes
                        for c in categoriesList:

                            # Merge nodes
                            query_string = "MERGE (e:Entity {Type:\"autoID\", Name:\'"+c+"\'})"
                            conn.query(query_string, db='neo4j')

                            # Create relationship
                            query_string = "MATCH (p:Post), (e:Entity) WHERE p.Link = \""+url+"\" AND p.Type = \""+pageType+"Original\" AND e.Type = \"autoID\" AND e.Name =\""+c+"\" CREATE (p)-[r:MENTIONED]->(e)"
                            conn.query(query_string, db='neo4j')

                elif pageType == 'qq':

                    # Brandwatch API does not provide QQ authors for some reason, and ID is only for the post
                    # So we're just going to do post and mentions

                    # Merge post node
                    query_string = "MERGE (p:Post {Type:\""+pageType+"\", Date:\""+date+"\", Link:\""+url+"\", Language:\""+language+"\", Text:\""+text+"\"})"
                    conn.query(query_string, db='neo4j')

                    # Create categories list nodes
                    for c in categoriesList:

                        # Merge nodes
                        query_string = "MERGE (e:Entity {Type:\"autoID\", Name:\'"+c+"\'})"
                        conn.query(query_string, db='neo4j')

                        # Create relationship
                        query_string = "MATCH (p:Post), (e:Entity) WHERE p.Link = \""+url+"\" AND p.Type = \""+pageType+"\" AND e.Type = \"autoID\" AND e.Name =\""+c+"\" CREATE (p)-[r:MENTIONED]->(e)"
                        conn.query(query_string, db='neo4j')

                elif pageType == 'tumblr':
                    title = str(f['title']) 
                    blogName = str(f['blogName'])

                    # Merge author node
                    query_string = "MERGE (e:Entity {Name:\'"+author+"\', Type:\"TumblrAuthor\"})"                
                    conn.query(query_string, db='neo4j')                          

                    # Merge tumblr blog
                    query_string = "MERGE (e:Entity {Name:\'"+blogName+"\', Type:\""+pageType+"Blog\", Location:\""+country+"\"})"                
                    conn.query(query_string, db='neo4j')                           

                    # Merge post
                    query_string = "MERGE (p:Post {Type:\""+pageType+"Post\", Name:\""+title+"\", Date:\""+date+"\", Link:\""+url+"\", Language:\""+language+"\", Text:\""+text+"\"})"
                    conn.query(query_string, db='neo4j') 

                    # Create author-post relationship           
                    query_string = "MATCH (e:Entity), (p:Post) WHERE e.Name = \""+author+"\" AND e.Type = \"TumblrAuthor\" AND p.Link = \""+url+"\" CREATE (e)-[r:AUTHORED]->(p)"
                    conn.query(query_string, db='neo4j')                

                    # Create blog-post relationship
                    query_string = "MATCH (e:Entity), (p:Post) WHERE e.Name = \""+blogName+"\" AND e.Type = \""+pageType+"Blog\" AND p.Link = \""+url+"\" CREATE (e)-[r:PUBLISHED]->(p)"
                    conn.query(query_string, db='neo4j')

                    # Create categories list nodes
                    for c in categoriesList:

                        # Merge nodes
                        query_string = "MERGE (e:Entity {Type:\"autoID\", Name:\'"+c+"\'})"
                        conn.query(query_string, db='neo4j')

                        # Create relationship
                        query_string = "MATCH (p:Post), (e:Entity) WHERE p.Link = \""+url+"\" AND p.Type = \""+pageType+"Post\" AND e.Type = \"autoID\" AND e.Name =\""+c+"\" CREATE (p)-[r:MENTIONED]->(e)"
                        conn.query(query_string, db='neo4j')

                elif pageType == 'forum':

                    title = str(f['title'])

                    # Merge forum node
                    query_string = "MERGE (e:Entity {Name:\'"+domain+"\', Type:\""+pageType+"\", Location:\""+country+"\"})"                
                    conn.query(query_string, db='neo4j')

                    # Merge post node
                    query_string = "MERGE (p:Post {Type:\""+pageType+"\", Name:\""+title+"\", Date:\""+date+"\", Link:\""+url+"\", Language:\""+language+"\", Text:\""+text+"\"})"
                    conn.query(query_string, db='neo4j')                

                    # Merge author node
                    query_string = "MERGE (e:Entity {Name:\'"+author+"\', Type:\"ForumAuthor\"})"                
                    conn.query(query_string, db='neo4j')                

                    # Create forum-post relationship
                    query_string = "MATCH (e:Entity), (p:Post) WHERE e.Name = \""+domain+"\" AND e.Type = \""+pageType+"\" AND p.Link = \""+url+"\" CREATE (e)-[r:PUBLISHED]->(p)"
                    conn.query(query_string, db='neo4j')

                    # Create author-post relationship
                    query_string = "MATCH (e:Entity), (p:Post) WHERE e.Name = \""+author+"\" AND e.Type = \"ForumAuthor\" AND p.Link = \""+url+"\" CREATE (e)-[r:AUTHORED]->(p)"
                    conn.query(query_string, db='neo4j')

                    # Create categories list nodes
                    for c in categoriesList:

                        # Merge nodes
                        query_string = "MERGE (e:Entity {Type:\"autoID\", Name:\'"+c+"\'})"
                        conn.query(query_string, db='neo4j')

                        # Create relationship
                        query_string = "MATCH (p:Post), (e:Entity) WHERE p.Link = \""+url+"\" AND p.Type = \""+pageType+"\" AND p.Date = \""+date+"\" AND e.Type = \"autoID\" AND e.Name =\""+c+"\" CREATE (p)-[r:MENTIONED]->(e)"
                        conn.query(query_string, db='neo4j')

                elif pageType == 'facebook':
                    threadID = f['threadId']
                    threadURL = f['threadURL']
                    threadAuthor = f['threadAuthor']
                    threadTitle = f['title'] # Yes, this is duplication, but it keeps my CQL easier to understand
                    threadDate = f['threadCreated']

                    if f['threadEntryType'] == 'reply':

                        # Merge original post node
                        # Note that original thread text doesn't come through on replies, so we're leaving that out
                        query_string = "MERGE (p:Post {ID:\""+threadID+"\", Type:\""+pageType+"Original\"})"
                        conn.query(query_string, db='neo4j')

                        query_string = "MATCH (p:Post {ID:\""+threadID+"\, Type:\""+pageType+"Original\"}) SET p.Name = \""+threadTitle+"\", p.Date = \""+threadDate+"\", p.Link = \""+threadURL+"\""
                        conn.query(query_string, db='neo4j')

                        # Merge reply post node
                        query_string = "MERGE (p:Post {Type:\""+pageType+"Reply\", Date:\""+date+"\", Link:\""+url+"\", Language:\""+language+"\", Text:\""+text+"\"})"
                        conn.query(query_string, db='neo4j')   

                        # Merge author node
                        query_string = "MERGE (e:Entity {Name:\'"+author+"\', Type:\"FacebookAuthor\"})"                
                        conn.query(query_string, db='neo4j') 
                        query_string = "MATCH (e:Entity {Name:\""+author+"\", Type:\"FacebookAuthor\"}) SET e.Country = \"" + country +"\""                   
                        conn.query(query_string, db='neo4j')

                        # Create reply-original relationship
                        query_string = "MATCH (c:Post), (o:Post) WHERE c.Link = \""+url+"\" AND c.Type = \""+pageType+"\"Reply\" AND o.ID = \""+threadID+"\" AND o.Type = \""+pageType+"Original\" CREATE (c)-[r:REPLIED_TO]->(o)" 
                        conn.query(query_string, db='neo4j')

                        # Create author-reply relationship
                        query_string = "MATCH (e:Entity), (c:Post) WHERE e.Name = \""+author+"\" AND e.Type = \"FacebookAuthor\" AND c.Link = \""+url+"\" AND c.Type = \""+pageType+"\"Reply\" CREATE (e)-[r:AUTHORED]->(c)"
                        conn.query(query_string, db='neo4j')

                        # Create categories list nodes
                        for c in categoriesList:

                            # Merge nodes
                            query_string = "MERGE (e:Entity {Type:\"autoID\", Name:\'"+c+"\'})"
                            conn.query(query_string, db='neo4j')

                            # Create relationship
                            query_string = "MATCH (p:Post), (e:Entity) WHERE p.Link = \""+url+"\" AND p.Type = \""+pageType+"Reply\" AND p.Date = \""+date+"\" AND e.Type = \"autoID\" AND e.Name =\""+c+"\" CREATE (p)-[r:MENTIONED]->(e)"
                            conn.query(query_string, db='neo4j')

                    elif f['threadEntryType'] == 'post':
                        # Merge original post node
                        # The original thread text does come through on original posts, so we can include it this time

                        query_string = "MERGE (p:Post {ID:\""+threadID+"\", Type:\""+pageType+"Original\"})"
                        conn.query(query_string, db='neo4j')

                        query_string = "MATCH (p:Post {ID:\""+threadID+"\, Type:\""+pageType+"Original\"}) SET p.Name = \""+threadTitle+"\", p.Date = \""+threadDate+"\", p.Link = \""+threadURL+"\", p.Text = \""+text+"\""
                        conn.query(query_string, db='neo4j')                    

                        # Merge author node
                        query_string = "MERGE (e:Entity {Name:\'"+author+"\', Type:\"FacebookAuthor\"})"                
                        conn.query(query_string, db='neo4j') 

                        query_string = "MATCH (e:Entity {Name:\""+author+"\", Type:\"FacebookAuthor\"}) SET e.Country = \"" + country +"\""                   
                        conn.query(query_string, db='neo4j')

                        # Create author-post relationship
                        query_string = "MATCH (e:Entity), (o:Post) WHERE e.Name = \""+author+"\" AND e.Type = \"FacebookAuthor\" AND o.ID = \""+threadID+"\" AND o.Type = \""+pageType+"Original\" CREATE (e)-[r:AUTHORED]->(o)"
                        conn.query(query_string, db='neo4j')                    

                        # Create categories list nodes
                        for c in categoriesList:

                            # Merge nodes
                            query_string = "MERGE (e:Entity {Type:\"autoID\", Name:\'"+c+"\'})"
                            conn.query(query_string, db='neo4j')

                            # Create relationship
                            query_string = "MATCH (p:Post), (e:Entity) WHERE p.Link = \""+url+"\" AND p.Type = \""+pageType+"Original\" AND p.Date = \""+threadDate+"\" AND e.Type = \"autoID\" AND e.Name =\""+c+"\" CREATE (p)-[r:MENTIONED]->(e)"
                            conn.query(query_string, db='neo4j')

                    else:
                        # My test data for Facebook is low
                        # So I'm putting this in here to handle anything unexpected

                        pass

                elif pageType == 'review':

                    # Reviews are funky and inconsistent across sites
                    # So we're going to handle this in a generalizable way that unfortunately leaves out thread relations
                    fullText = f['itemReview'] + ' ' +text # Putting the extra review text in the same field with rest of the text

                    # Generate an id for the post since Brandwatch doesn't provide fields guaranteed to be unique
                    # Although this theoretically could result in duplicates, the odds are small enough for this project
                    # For production, we may want to do a CQL query to confirm
                    randID = randint(1, 999999999) 
                    randID = str(randID)

                    # Merge review node
                    query_string = "MERGE (p:Post {Type:\""+pageType+"\", ID:\""+randID+"\", Date:\""+date+"\", Link:\""+url+"\", Language:\""+language+"\", Text:\""+fullText+"\"})"
                    conn.query(query_string, db='neo4j')

                    # Merge review site node
                    query_string = "MERGE (e:Entity {Name:\'"+domain+"\', Type:\""+pageType+"\", Location:\""+country+"\"})"                
                    conn.query(query_string, db='neo4j')

                    # Merge author node
                    query_string = "MERGE (e:Entity {Name:\'"+author+"\', Type:\"ReviewAuthor\"})"                
                    conn.query(query_string, db='neo4j')

                    # Create forum-post relationship
                    query_string = "MATCH (e:Entity), (p:Post) WHERE e.Name = \""+domain+"\" AND e.Type = \""+pageType+"\" AND p.ID = \""+randID+"\" CREATE (e)-[r:PUBLISHED]->(p)"
                    conn.query(query_string, db='neo4j')

                    # Create author-post relationship
                    query_string = "MATCH (e:Entity), (p:Post) WHERE e.Name = \""+author+"\" AND e.Type = \"ReviewAuthor\" AND p.ID = \""+randID+"\" CREATE (e)-[r:AUTHORED]->(p)"
                    conn.query(query_string, db='neo4j')

                    # Create categories list nodes
                    for c in categoriesList:

                        # Merge nodes
                        query_string = "MERGE (e:Entity {Type:\"autoID\", Name:\'"+c+"\'})"
                        conn.query(query_string, db='neo4j')

                        # Create relationship
                        query_string = "MATCH (p:Post), (e:Entity) WHERE p.ID = \""+randID+"\" AND p.Type = \""+pageType+"\" AND e.Type = \"autoID\" AND e.Name =\""+c+"\" CREATE (p)-[r:MENTIONED]->(e)"
                        conn.query(query_string, db='neo4j')

                else:
                    # There are several other item types, but they're extremely rare 
                    # So we're just going to ignore them

                    pass
        except Exception as e:
            print(e)
            print(pageType)
            print("failed")
            pass
            
    counter += 1

0
1
Retweeted
2
3
Retweeted
4
5
Retweeted
6
Retweeted
7
8
Retweeted
9
Regular
10
11
Retweeted
12
Regular
13
Quote tweet
14
Query failed: {code: Neo.DatabaseError.General.UnknownError} {message: Lexical error at line 1, column 43.  Encountered: <EOF> after : ""}
15
16
17
Query failed: {code: Neo.ClientError.Statement.SyntaxError} {message: Invalid input 'С': expected whitespace, '.', node labels or rel types, '[', '^', '*', '/', '%', '+', '-', "=~", IN, STARTS, ENDS, CONTAINS, IS, '=', '~', "<>", "!=", '<', '>', "<=", ">=", AND, XOR, OR, ',' or '}' (line 1, column 43 (offset: 42))
"MERGE (p:Post {Type:"news", Name:"Звезда "Сумерек" Кристен Стюарт обручилась со своей девушкой", Date:"2021-11-02T16:44:41.000+0000", Link:"https://focus.ua/lifestyle/496885-zvezda-sumerek-kristen-styuart-obruchilas-so-svoey-devushkoy", Language:"ru", Text:"...фильму Сумерки Робертом Паттинсоном. Однако ему она изменила с режиссером фильма Белоснежка и охотник Рупертом Сандерсом. Кристен Стюарт и Роберт Патти

376
377
378
379
380
381
382
383
Regular
384
385
386
387
388
389
390
391
392
393
Regular
394
395
396
397
398
Reply
399
Retweeted
400
401
Regular
402
403
404
Retweeted
405
Regular
406
Retweeted
407
408
409
Reply
410
411
Regular
412
413
414
Regular
415
Quote tweet
416
Regular
417
418
419
420
421
422
Regular
423
Quote tweet
424
425
Retweeted
426
Retweeted
427
428
429
Regular
430
431
432
433
434
435
436
437
Regular
438
439
440
441
442
443
444
Quote tweet
445
Retweeted
446
447
448
449
450
451
452
453
Retweeted
454
455
456
457
Quote tweet
458
459
460
461
462
463
Retweeted
464
465
466
Retweeted
467
468
469
470
Quote tweet
471
Regular
472
Quote tweet
473
474
Regular
475
476
477
478
479
480
481
482
483
484
Retweeted
485
486
487
488
489
490
491
492
Regular
493
494
Reply
495
496
497
Regular
498
Regular
499
Retweeted
500
501
502
503
504
505
506
507
Quote tweet
508
509
Regular
510
511
Reply
512
513
Reply
514
515
516
Regular
517
518
Retweeted
519
Retweeted
520
521
522
Retweeted
523
Retweeted
524
Regu

Reply
835
Regular
836
837
838
Regular
839
840
841
842
843
844
845
846
847
848
849
Query failed: {code: Neo.ClientError.Statement.SyntaxError} {message: Invalid input 'G': expected whitespace, '.', node labels or rel types, '[', '^', '*', '/', '%', '+', '-', "=~", IN, STARTS, ENDS, CONTAINS, IS, '=', '~', "<>", "!=", '<', '>', "<=", ">=", AND, XOR, OR, ',' or '}' (line 1, column 36 (offset: 35))
"MERGE (p:Post {Type:"blog", Name:""Gen: Lock" on HBO Max", Date:"2021-11-01T18:35:00.000+0000", Link:"http://www.sneakpeek.ca/2021/11/gen-lock-on-hbo-max.html", Language:"en", Text:"Season Two of the science fiction anime TV series Gen:Lock, created by Gray Haddock, starring the voices of Michael B. Jordan, Dakota Fanning, Golshifteh Farahani, Maisie Williams, Kōichi Yamadera, Asia Kate Dillon, Blaine Gibson, Gray Haddock, Miles Luna, Chad James, Monica Rial and David Tennant, streams November 4, 2021 on HBO Max: ...'Gen:Lock' is set in a dystopian future Earth where an international coalition 

Query failed: {code: Neo.ClientError.Statement.SyntaxError} {message: Invalid input 'G': expected whitespace, '.', node labels or rel types, '[', '^', '*', '/', '%', '+', '-', "=~", IN, STARTS, ENDS, CONTAINS, IS, '=', '~', "<>", "!=", '<', '>', "<=", ">=", AND, XOR, OR, FROM GRAPH, USE GRAPH, CONSTRUCT, LOAD CSV, START, MATCH, UNWIND, MERGE, CREATE UNIQUE, CREATE, SET, DELETE, REMOVE, FOREACH, WITH, CALL, RETURN, UNION, ';' or end of input (line 1, column 117 (offset: 116))
"MATCH (p:Post), (e:Entity) WHERE p.Link = "http://www.sneakpeek.ca/2021/11/gen-lock-on-hbo-max.html" AND p.Name = ""Gen: Lock" on HBO Max" AND e.Type = "autoID" AND e.Name ="Asia Kate Dillon" CREATE (p)-[r:MENTIONED]->(e)"
                                                                                                                     ^}
Query failed: {code: Neo.ClientError.Statement.SyntaxError} {message: Invalid input 'G': expected whitespace, '.', node labels or rel types, '[', '^', '*', '/', '%', '+', '-', 

Query failed: {code: Neo.ClientError.Statement.SyntaxError} {message: Invalid input 'G': expected whitespace, '.', node labels or rel types, '[', '^', '*', '/', '%', '+', '-', "=~", IN, STARTS, ENDS, CONTAINS, IS, '=', '~', "<>", "!=", '<', '>', "<=", ">=", AND, XOR, OR, FROM GRAPH, USE GRAPH, CONSTRUCT, LOAD CSV, START, MATCH, UNWIND, MERGE, CREATE UNIQUE, CREATE, SET, DELETE, REMOVE, FOREACH, WITH, CALL, RETURN, UNION, ';' or end of input (line 1, column 117 (offset: 116))
"MATCH (p:Post), (e:Entity) WHERE p.Link = "http://www.sneakpeek.ca/2021/11/gen-lock-on-hbo-max.html" AND p.Name = ""Gen: Lock" on HBO Max" AND e.Type = "autoID" AND e.Name ="Weller" CREATE (p)-[r:MENTIONED]->(e)"
                                                                                                                     ^}
Query failed: {code: Neo.ClientError.Statement.SyntaxError} {message: Invalid input 'G': expected whitespace, '.', node labels or rel types, '[', '^', '*', '/', '%', '+', '-', "=~", IN, 

...

11959
11960
11961
11962
Regular
11963
11964
Query failed: {code: Neo.ClientError.Statement.SyntaxError} {message: Invalid input ' ': expected 't/T' (line 1, column 45 (offset: 44))
"MERGE (e:Entity {Type:"autoID", Name:'Divi's Laboratories Ltd.'})"
                                             ^}
11965
11966
Retweeted
11967
Regular
11968
Retweeted
11969
Reply
11970
11971
11972
11973
11974
Regular
11975
Retweeted
11976
Query failed: {code: Neo.ClientError.Statement.SyntaxError} {message: Invalid input ''': expected 't/T' (line 1, column 58 (offset: 57))
"MERGE (e:Entity {Type:"autoID", Name:'Michael B. Jordan's'})"
                                                          ^}
11977
Reply
11978
11979
Retweeted
11980
Retweeted
11981
11982
11983
11984
11985
11986
Reply
11987
Retweeted
11988
Retweeted
11989
11990
11991
Regular
11992
11993
11994
11995
11996
Retweeted
11997
11998
11999
Retweeted
12000
Retweeted
12001
12002
12003
Retweeted
12004
12005
Reply
12006
12007
12008
12009
12010
Regular


12185
Reply
12186
Retweeted
12187
12188
12189
12190
Retweeted
12191
12192
Retweeted
12193
Retweeted
12194
12195
Regular
12196
12197
Retweeted
12198
12199
12200
12201
12202
Retweeted
12203
Retweeted
12204
12205
Regular
12206
12207
12208
Reply
12209
12210
Retweeted
12211
12212
Retweeted
12213
12214
12215
12216
12217
Retweeted
12218
12219
12220
Retweeted
12221
12222
Reply
12223
Retweeted
12224
12225
12226
12227
Retweeted
12228
12229
Retweeted
12230
Retweeted
12231
Retweeted
12232
12233
Retweeted
12234
12235
Retweeted
12236
12237
12238
Retweeted
12239
12240
Retweeted
12241
12242
12243
Retweeted
12244
12245
Retweeted
12246
Regular
12247
12248
12249
Retweeted
12250
Retweeted
12251
Retweeted
12252
12253
12254
12255
Quote tweet
12256
12257
12258
12259
12260
Retweeted
12261
12262
12263
Retweeted
12264
12265
12266
12267
12268
Retweeted
12269
Retweeted
12270
Retweeted
12271
12272
12273
12274
12275
12276
12277
12278
12279
Retweeted
12280
12281
12282
12283
12284
Retweeted
12285
12286
12287
Retweete

12604
Retweeted
12605
12606
Regular
12607
12608
12609
Retweeted
12610
Retweeted
12611
12612
12613
Query failed: {code: Neo.ClientError.Statement.SyntaxError} {message: Invalid input 'L': expected whitespace, '.', node labels or rel types, '[', '^', '*', '/', '%', '+', '-', "=~", IN, STARTS, ENDS, CONTAINS, IS, '=', '~', "<>", "!=", '<', '>', "<=", ">=", AND, XOR, OR, ',' or '}' (line 1, column 73 (offset: 72))
"MERGE (p:Post {Type:"news", Name:"Santa Fe: piden procesar al líder de "Los Monos" por el secuestro del hijo de un empresario", Date:"2021-10-22T13:05:43.000+0000", Link:"https://carmendearecodigital.com.ar/contenido/12183/santa-fe-piden-procesar-al-lider-de-los-monos-por-el-secuestro-del-hijo-de-un-em", Language:"es", Text:"..., de Villa Gobernador Gálvez-, donde permaneció cautiva unas dos horas, periodo en que los secuestradores se comunicaron con sus padres y les exigieron dinero por su liberación. Tras el pago de 10 mil dólares y joyas que fueron entregadas en las inmediaci

Regular
12835
12836
12837
12838
12839
12840
12841
12842
12843
12844
12845
Regular
12846
12847
12848
Regular
12849
Retweeted
12850
12851
Regular
12852
Retweeted
12853
Regular
12854
12855
12856
12857
12858
12859
12860
12861
12862
12863
Retweeted
12864
12865
Retweeted
12866
Regular
12867
12868
12869
12870
12871
12872
12873
Regular
12874
12875
12876
12877
Query failed: {code: Neo.ClientError.Statement.SyntaxError} {message: Invalid input 'L': expected whitespace, comment, '.', node labels or rel types, '[', '^', '*', '/', '%', '+', '-', "=~", IN, STARTS, ENDS, CONTAINS, IS, '=', '~', "<>", "!=", '<', '>', "<=", ">=", AND, XOR, OR, ',' or '}' (line 1, column 47 (offset: 46))
"MERGE (e:Entity {Type:"autoID", Name:'Land O' Lakes'})"
                                               ^}
12878
12879
Reply
12880
12881
12882
Regular
12883
Regular
12884
Retweeted
12885
Retweeted
12886
12887
Retweeted
12888
Retweeted
12889
Retweeted
12890
Retweeted
12891
Retweeted
12892
Retweeted
12893
Retweeted
12894


Retweeted
13206
13207
13208
Regular
13209
Query failed: {code: Neo.ClientError.Statement.SyntaxError} {message: Invalid input 'F': expected whitespace, '.', node labels or rel types, '[', '^', '*', '/', '%', '+', '-', "=~", IN, STARTS, ENDS, CONTAINS, IS, '=', '~', "<>", "!=", '<', '>', "<=", ">=", AND, XOR, OR, ',' or '}' (line 1, column 68 (offset: 67))
"MERGE (p:Post {Type:"forum", Name:"So as I suspected, majority of "Farms" are just giant income and tax reduction shcemes.", Date:"2021-10-22T00:04:00.000+0000", Link:"https://www.ar15.com/forums/General/So-as-I-suspected-majority-of-Farms-are-just-giant-income-and-tax-reduction-shcemes-/5-2497675/?page=5#i95436890#9e030d5965bf44814b8f628e3e625a0c", Language:"en", Text:"Quote History Originally Posted By Miami_JBT: The most productive cow-calf ranch in the country is the Deseret Ranch in FL. They're corporate owned by the Mormon Church. View Quote Ok. Still waiting on the guy to cite that half of all cereal grains are produced by agr